# 내비게이션 기능

- 내비게이션바는 모든 페이지에서 보여야 하므로 base.html 템플릿 파일에서 추가
- 부트스트랩 내비게이션 바는 웹 브라우저의 크기가 줄어들면 메뉴 버튼이 생기면서 계정생성과 로그인 링크는 사라짐
    - 지금은 메뉴 버튼을 클릭해도 변화가 없음
    - 부트스트랩 자바스크립트 파일이 base.html에 포함되지 않았기 때문에

## include로 내비게이션 바 추가

- include 기능은 템플릿의 특정 영역을 중복, 반복해서 사용할 경우에 유용
    - 중복, 반복하는 템플릿의 특정 영역을 따로 템플릿 파일로 만들고
    - include 기능으로 그 템플릿을 포함
- navbar.html 파일은 base.html에서 한 번만 사용되지만 따로 파일로 관리해야 이후 유지 보수하는데 유리하기 때문에 분리

# 게시판 페이징 기능

- 페이징 기능이 없으면 게시물이 많아졌을 때 한 화면에 모든 게시물을 다 표시하게 됨
- 이런 경우 스크롤 바를 계속 내려야하는 등의 불편함이 생김

# 페이징 기능 구현

- 플라스크에서는 paginate 함수를 사용하여 쉽게 페이징 기능을 구현할 수 있음
- question_list = question_list.paginate(page=page, per_page=10)
    - question_list 데이터에 paginate 함수로 페이징을 적용
    - page는 현재 조회할 페이지의 번호를 의미
    - per_page는 페이지마다 보여줄 게시물 수
- paginate 함수는 조회한 데이터를 감싸 Pagination 객체로 반환
- Pagination 객체의 속성
    - items: 현재 페이지에 해당하는 게시물 리스트
    - total: 게시물 전체 개수
    - per_page: 페이지당 보여줄 게시물 개수
    - iter_pages: 페이지 범위
    - prev_num/next_num: 이전 페이지 번호 / 다음 페이지 번호
    - has_prev/has_next: 이전 페이지 존재 여부 / 다음 페이지 존재 여부(True/False)

# 템플릿 필터 생성

- 템플릿 필터는 {{ question.answer_set|length }}와 같이 템플릿에서 사용한 객체에 파이프라인 문자(|)를 붙여 필터 기능을 수행

## 템플릿 필터를 사용해 작성일시 수정

- 현재 질문 목록 페이지에서는 시간이 지나치게 자세하게 표시됨
- datetime 객체를 문자열로 만드는 템플릿 필터 생성(filter.py)
- 필터 적용
    - app.jinja_env.filters["datetime"] = format_datetime
        - datetime이라는 이름으로 format_datetime 필터를 등록

# 템플릿필터 사용

- {{ question.create_date|datetime }}
    - create_date 데이터에 datetime 필터를 적용
        - datetime 필터는 format_datetime 필터
        - format_datetime의 매개변수 value에 question.create_date가 전달됨

# 게시물에 일련번호 추가

- 현재 질문 목록 조회 화면에서는 각 페이지마다 페이지 번호가 항상 1부터 시작하는 문제가 있음
- 게시물 번호 공식: 번호 = 전체 게시물 개수 - ((현재 페이지-1)\*페이지당 게시물 개수) - 나열 인덱스(0 ~ 9)
    - 번호: 최종 표시될 게시물 번호
    - 전체 게시물 개수: 데이터베이스에 저장된 게시물 전체 개수
    - 현재 페이지: 페이징에서 현재 선택한 페이지
    - 페이지당 게시물 개수: 페이징에서 현재 선택한 페이지의 게시물 개수
    - 나열 인덱스: for문 안의 게시물 순서
    
- {{ question_list.total - ((question_list.page - 1) * question_list.per_page) - loop.index0 }}
    - question_list.total: 전체 게시물 개수
    - question_list.page: 현재 페이지
    - question_list.per_page: 페이지당 게시물 개수
    - loop.index0: 나열 인덱스(0부터 시작_ 0 붙여주지 않으면 1부터 시작)

# 질문 목록에서 질문에 달린 답변 개수 표시 

# 회원가입

- 지금까지는 질문, 답변 관련 모델만 사용했지만
- 회원가입 기능을 추가하기 위해서는 회원 정보를 위한 모델이 필요함
- 회원 정보 모델 구성
    - username : 사용자 이름(ID)
    - password : 비밀번호
    - email : 이메일

# 회원 정보 모델 생성

- 모델 생성 순서
    1. models.py에 모델 작성
    2. flask db migrate
    3. flask db upgrade

## 회원가입 폼 생성

- email 필드에 설정한 이메일 검증을 사용하려면 email-validator를 설치해야함
    - $ pip install email_validator

## 회원가입 화면 개발

### 회원가입 뷰 작성

- /auth/ 접두어로 시작하는 url이 호출되면 auth_view.py 파일들이 호출될 수 있도록 블루프린트 auth를 추가
- 회원가입을 위한 signup 함수를 생성
- signup 함수는 POST 방식 요청에는 계정 등록, GET 방식 요청에는 계정 등록 템플릿 렌더링을 수행
- username으로 데이터를 조회해서 이미 등록된 사용자인지 확인
    - 만약 이미 등록된 사용자라면 flash()로 오류를 발생시킴
- 비밀번호는 입력받은 값을 generate_password_hash 함수로 암호화하여 저장

# 템플릿 작성

- 회원가입을 위한 사용자이름, 비밀번호, 비밀번호확인, 이메일에 해당되는 데이터를 입력하고 생성하기 버튼을 누르면 폼 데이터가 POST 방식으로 /auth/signup/url에 전송됨
- {% include "form_errors.html" %}
    - 회원가입을 할 때 발생하는 오류를 표시하는 템플릿

### form_errors.html 작성

- form_errors.html 파일은 필드에서 발생한 오류와 flash를 거치면서 발생한 오류를 표시
    - 필드 오류: 폼 validators 검증에 실패한 경우
    - flash 오류: flash("이미 존재하는 사용자입니다.")와 같은 로직으로 표시되는 경우

# 로그인 / 로그아웃

- 게시판은 여러 사람이 사용하는 서비스이므로 질문한 사람, 답변한 사람을 구별하는 로그인, 로그아웃 기능은 필수 기능임

## 로그인

### 로그인 폼

### 로그인 라우트 함수 생성

- /auth/login/ 에 매핑되는 login 함수를 생성
    - signup 함수와 비슷한 패턴
    - POST 방식 요청에는 로그인을 수행, GET 방식 요청에는 로그인 템플릿을 렌더링
- 로그인 작업 수행 과정
    - 폼 입력으로 받은 username으로 데이터베이스에 해당 사용자가 있는지를 검사
        - 만약 사용자가 없으면 "존재하지 않는 사용자입니다."라는 오류 발생
    - 사용자가 존재한다면 폼 입력으로 받은 password와 check_password_hash 함수를 사용해서 데이터베이스의 비밀번호와 일치하는지를 비교
    - 사용자도 존재하고 비밀번호도 올바르다면 플라스크 세션(session)에 키와 키값을 저장
        - 키 : user_id
        - 키값 : 데이터베이스에서 조회된 사용자의 id값
- 세션(session)
    - 플라스크가 자동으로 생성하여 제공하는 변수
    - 플라스크 서버를 구동하는 동안에 영구히 참조할 수 있는 값
        - session 변수에 user의 id값을 저장했으므로 다양한 url 요청에서 이 세션값을 사용할 수 있음
        - 예) 현재 웹 브라우저를 요청한 주체가 로그인한 사용자인지 아닌지도 판단할 수 있음

# 로그인 템플릿 생성

- 로그인 폼에서 생성한 필드 2개(username, password)를 input태그로 작성

## 내비게이션 바에 로그인 링크 추가

- 로그인 링크를 누르면 로그인 화면이 나타나고 로그인 기능이 작동함
- 지금은 로그인한 후에도 내비게이션 바에는 여전히 로그인 링크가 남아있음
    - 로그인 후에는 로그인 링크가 로그아웃 링크로 바뀌어야 함
- 사용자의 로그인 여부는 session에 저장된 값을 조사하면 알 수 있음

### 로그인한 사용자 정보 조회 함수 구현

- @bp.before_app_request
    - 이 데코레이터가 사용된 함수는 라우트 함수보다 먼저 실행됨
    - 즉, 앞으로 load_logged_in_user 함수는 모든 라우트 함수보다 먼저 실행됨
- g
    - 플라스크가 제공하는 컨텍스트 변수
    - request 변수처럼 요청 -> 응답 과정에서 유효함
- load_logged_in_user 함수에서는 session 변수에 user_id 값이 있으면 데이터베이스에서 데이터를 조회하여 g.user에 저장
    - 이렇게 하면 로그인 검사를 할 때 session을 조사할 필요가 없고 g.user에 값이 있는지만 알아내면 됨
    - 또한 g.user에는 User 객체가 저장되어 있으므로 여러가지 상세 사용자 정보를 추가로 확인할 수 있다는 이점이 있음

### 내비게이션바 수정

- g.user는 load_logged_in_user 함수로 생성한 사용자 정보값
    - 로그인 되어 있다면 g.user가 만들어진 상태이므로 username의 값과 로그아웃 링크를 보여줌
    - 로그인 되어 있지 않다면 로그인과 계정생성 링크를 보여줌

# 로그아웃

### 로그아웃 함수 구현

- session.clear()
    - 세션의 모든 값을 삭제
    - 세션에 저장된 user_id는 삭제되며, load_logged_in_user 함수에서도 session값을 읽을 수 없게 되므로 g.user는 None이 됨

# 모델 수정

- 회원가입, 로그인, 로그아웃 기능이 완성되어 질문, 답변을 누가 작성했는지 알 수 있게 되었음
- 이제 Question, Answer 모델에 글쓴이 필드를 추가할 수 있음

## SQLite 설정 수정

- SQLite 데이터베이스는 다른 DB와는 다르게 naming_convention을 따로 설정해야 함

## Question 모델에 필드 추가

- Question 모델에 작성자 필드 추가
- user_id : User 모델 데이터의 id값을 Question 모델에 포함시키기 위함
- user : Quesiton 모델에서 User 모델을 참조하기 위한 필드
    - backref를 설정하여 User 모델에서도 Question모델 데이터를 참조할 수 있음
- user_id 필드는 Null을 허용하지 않는데, 미리 만들어둔 데이터는 user_id 필드의 값이 없기 때문에 오류가 발생함

## flask db upgrade 오류 해결

- 모델에 데이터가 있는데 nullable=False인 필드를 추가할 때 항상 거쳐야 하는 작업
    1. user_id의 nullable 설정을 True로 변경
    2. user_id를 임의의 값으로 설정(예:1)
    3. flask db migrate, flask db upgrade를 실행
    4. user_id의 nullable 설정을 False로 변경
    5. flask db migrate, flask db upgrade를 실행
- 설계가 아주아주 잘못된 경우에 해당함

### nullable 설정을 True로 변경

- server_default="1"
    - "1" : 최초로 생성한 User 모델 데이터의 id
    - 이미 저장된 데이터의 user_id 값을 설정하기 위해서 server_default를 사용
- server_default와 default의 차이
    - server_default는 flask db upgrade를 수행할 때 해당 필드를 갖고 있지 않던 기존 데이터도 모두 기본값을 적용
    - default는 새로 추가되는 데이터에만 기본값을 적용

## flask db migrate

- 다시 flask db migrate 명령을 수행하면 오류가 발생함
    - 이전의 migrate 명령은 제대로 수행되었지만 upgrade를 실패하여 정상적으로 종료되지 않았기 때문

### flask db heads

- flask db heads: 현재 migrate 작업을 진행하는 최종 리비전 확인

### flask db current

- flask db current : 현재 리비전 확인
- 현재 시점 리비전과 최종 리비전이 일치해야 migrate 작업을 진행할 수 있음
    - 앞서 migrate 이후 upgrade를 실패했기 때문에 일치하지 않음

### 현재 리비전을 최종 리비전으로 변경

- flask db stamp heads: 최종 리비전과 현재 리비전이 같은 값으로 변경됨

### Question 모델 재수정

- Question 모델의 user_id 필드를 nullable=False로 변경, server_default 삭제

## Answer 모델에 필드 추가

- Answer 모델도 같은 방법으로 user_id 필드를 추가
    - nullable=False로 설정하면 데이터베이스가 꼬이기 때문에 nullable=True로 설정한 뒤 작업 진행

### Answer 모델 재수정

## 질문, 답변 등록 시 user필드 추가

- Question, Answer 모델에 user필드를 추가했으므로 질문, 답변 등록 기능에도 user 필드를 반영해야함


### 답변 등록 뷰 수정

- answer를 생성할 때 user=g.user를 추가
- g.user: auth_view.py의 @bp.before_app_request 데코레이터로 만든 로그인한 사용자 정보

## 로그인 상태가 필요한 함수 추가

- 로그아웃 상태에서 질문 또는 답변을 등록하려면 오류가 발생함
    - 로그아웃 했으므로 g.user의 값이 None이기 때문에
- 이 문제를 해결하려면 로그아웃 상태에서 질문 또는 답변을 등록할 때 사용자를 로그인 페이지로 리다이렉트 하는 방법이 있음
    - 이렇게 하려면 모든 질문, 답변 등록 함수의 시작 부분에 리다이렉트를 처리하기 위한 코드를 추가해야함
    - 하지만 이 방식은 같은 코드가 중복되므로 비효율적인 방식임

### 데코레이터 함수 생성

- 데코레이터는 기존 함수를 감싸는 방법으로 간단히 기능 구현 가능
- 다른 함수에 @login_required를 사용하면 login_required가 먼저 실행됨
- login_required : g.user가 있는지 조사하여, 없으면 로그인 url로 리다이렉트 하고, g.user가 있으면 원래 함수를 그대로 실행

## 데코레이터 적용

- 로그인이 필요한 함수에 @login_required 적용

# 로그아웃 상태에서 답변 등록 불가능하게 만들기

- 이제 로그아웃 상태에서는 답변 등록이 불가능함에도 불구하고 답변 등록을 할 수 있는 것처럼 되어 있음
    - 로그아웃 상태에서는 로그인 화면으로 리다이렉트 하지만 처음부터 글 작성을 시도할 수 없게 만드는 것이 더 자연스러움

# 글쓴이 표시 기능

- 앞서 Question 모델과 Answer모델에 user 필드를 추가했기 때문에 이제는 글쓴이 정보를 표시할 수 있음

## 질문 목록 조회 화면에서 글쓴이 표시

- text-center 클래스 : 가운데 정렬
- text-start 클래스 : 왼쪽 정렬
- style = "width : 50%" : 너비가 전체에서 50%를 차지하도록 설정

# 게시물 수정/삭제 기능

### 모델 수정

- Question, Answer 모델에 수정일자 필드 추가
    - 수정일자는 수정이 발생할 경우에만 생성되도록 null을 허용

### 질문 수정 기능

- 질문 수정 버튼 추가
    - 질문 수정 버튼은 로그인한 사용자와 글쓴이가 같은 경우에만 보여지도록 설정
- 질문 수정은 로그인이 필요함
- 만약 로그인한 사용자와 질문의 작성자가 다르면 수정할 수 없음
- modify 함수가 GET 방식으로 요청되는 경우는 질문수정 버튼을 눌렀을 경우
    - 이때 이미 수정할 질문에 해당하는 데이터가 보여야함
        - QuestionForm(obj=question) : 데이터베이스에서 조회한 데이터를 템플릿에 적용
- modify 함수가 POST 방식으로 요청되는 경우는 질문 수정 화면에서 데이터를 수정한 다음 저장하기 버튼을 누른 경우
    - form.validate_on_submit 함수에서 QuestionForm을 검증하여 아무 이상이 없으면 변경된 데이터를 저장
    - form.populate_obj(question):form 변수에 들어있는 데이터(화면에 입력되어 있는 데이터)를 question 객체에 적용

## 질문 삭제 기능

- 질문 삭제 버튼 추가
    - 삭제 기능에서 삭제 버튼을 구현할 때 확인창을 보여주도록 javascript를 사용하여 개발

# 답변 수정/삭제 기능

- 답변 수정&삭제는 질문 수정&삭제와 거의 비슷하지만
- 답변 수정은 답변 등록 템플릿이 없으므로 답변 수정에 사용할 템플릿이 추가로 필요

## 수정일시 표시

# 댓글 기능

## Comment 모델 생성

- Comment 모델의 구성
    - id: 댓글 고유 번호
    - user_id: 댓글 작성자(User 모델과 관계)
    - content: 댓글 내용
    - create_date: 댓글 작성 일시
    - modify_date: 댓글 수정 일시
    - question_id: 댓글의 질문(Question 모델과 관계)
    - answer_id: 댓글의 답변(Answer 모델과 관계)
- 질문에 댓글을 작성하면 question_id에 필드값이 저장되고, 답변에 댓글을 작성하면 answer_id에 필드값이 저장됨
    - 즉, Comment 모델의 데이터에는 question_id 또는 answer_id 필드 중 하나에만 값이 저장되므로 nullable=True여야 함

## 질문 댓글 기능 추가

- 질문에 등록된 댓글의 내용, 작성자, 작성일시, 수정일시를 출력
- 댓글 작성자와 로그인한 사용자가 같으면 "수정", "삭제" 링크가 보여짐
- 댓글을 작성할 수 있는 댓글 추가 링크 출력

### 질문 댓글 폼 작성

### 질문 댓글 등록 함수 개발

- board/views/comment_view.py 파일을 생성하여 함수 개발
- create_question 함수는 질문에 달린 댓글이므로 comment 데이터의 question 필드에 값을 설정하였음

## 블루프린트 등록

- board/\_\_init__.py에 comment_view를 import 한 후 app.register_blueprint()

## 질문 댓글 템플릿 작성

- board/templates/comment/comment_form.html

## 질문 댓글 수정 함수 개발

- board/views/comment_view.py

# 추천 기능

## Question, Answer 모델 변경_다대다 관계

- 추천은 질문이나 답변에 적용해야 하는 요소
- 그러려면 Question, Answer 모델에 추천인 필드를 추가해야 함
    - 1명은 여러 글에 추천을 할 수 있고
    - 1개의 글에는 여러 명이 추천할 수 있음
    - 이런 경우에는 다대다(Many To Many)관계를 사용해야함

# Question 모델에 voter 필드 추가

- voter 필드는 user 필드와 똑같이 User 모델의 relationship으로 만들지만 secondary 설정을 했다는 차이가 있음
    - secondary 설정은 voter가 다대다 관계이며, question_voter 테이블을 참조한다는 것을 설정

# 질문 추천 기능

- 질문 상세 조회 화면에서 질문 추천을 할 수 있도록 질문 추천 버튼 개발
- 부트스트랩의 row, col-1, col-11을 이용하여 추천 영역의 너비는 전체 너비의 1/12로, 질문 영역의 너비는 전체 너비의 11/12로 구성
    - 추천 영역에는 질문 추천 개수, 추천 버튼을 추가

### 추천 버튼 확인창 개발

- 추천 버튼을 눌렀을 때 정말로 추천하시겠습니까?라는 확인창이 나타나도록 기능 개발
- board/templates/question/question_detail.html
    - script 블록에 코드 추가